# Introducción

Una breve descripción del dataset, si hace falta.

# Preparación de datos

## Carga de datos y corrección de tipos

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Ajustar según lo necesario
df = pd.read_csv('')
df.head()

In [ ]:
df.info()

## Corrección de tipos de datos

In [ ]:
# Esto debería ser suficiente
object_columns = [col for col in df.columns if df[col].dtype == "object"]

for col in object_columns:
    df[col] = df[col].astype("category")
    print(f"df[{col}]: {df[col].unique()}")

## Descripción estadística de los datos

In [ ]:
df.describe()

### Histogramas

In [ ]:
numeric_columns = df.select_dtypes(include=[np.number]).columns

plt.figure(figsize=(20, 25))
for i, column in enumerate(numeric_columns):
    plt.subplot(10, 4, i + 1)
    plt.hist(df[column].dropna(), bins=30, alpha=0.7, edgecolor='black')
    plt.title(f'Histograma de {column}')
    plt.tight_layout()
plt.show()

### Boxplots

In [ ]:
plt.figure(figsize=(20, 25))
for i, column in enumerate(numeric_columns):
    plt.subplot(10, 4, i + 1)
    plt.boxplot(df[column].dropna())
    plt.grid()
    plt.title(f'Boxplot de {column}')
    plt.tight_layout()
plt.show()

### Gráficos de barras

In [ ]:
plt.figure(figsize=(20, 25))
for i, column in enumerate(object_columns):
    plt.subplot(10, 4, i + 1)
    plt.bar(df[column].unique(), df[column].value_counts())
    plt.title(f'Histograma de {column}')
    plt.tight_layout()
plt.show()

## Limpieza de atípicos

### Variables cuantitativas

**Nota: NO SIEMPRE ES NECESARIO IMPUTAR. ESTO DEPENDE DEL CONTEXTO.**

---

Para el caso de que solo sea necesario imputar una variable, se puede usar el código

```python
col = ""

Q1 = df[col].quantile(0.25)
Q3 = df[col].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
df[col] = np.where((df[col] < lower_bound) | (df[col] > upper_bound), np.nan, df[col])
```

De todos modos, el código suministrado abajo con el for debería ser suficiente.

In [ ]:
for col in numeric_columns:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df[col] = np.where((df[col] < lower_bound) | (df[col] > upper_bound), np.nan, df[col])

Imputando con la media,

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')
df[numeric_columns] = imputer.fit_transform(df[numeric_columns])
df.info()

Graficando los datos para verificar que se hayan eliminado los outliers:

In [ ]:
plt.figure(figsize=(20, 25))
for i, column in enumerate(numeric_columns):
    plt.subplot(10, 4, i + 1)
    plt.boxplot(df[column].dropna())
    plt.grid()
    plt.title(f'Histograma de {column}')
    plt.tight_layout()
plt.show()

### Variables cualitativas

In [ ]:
categorical_imputer = SimpleImputer(missing_values=np.nan, strategy="most_frequent")
df[object_columns] = categorical_imputer.fit_transform(df[object_columns])
df.info()

## Transformaciones de datos

### LabelEncoder

Aquí, se asigna a `objective_variable` el nombre de la variable objetivo a aplicarle el `LabelEncoder`. Si la variable objetivo es numérica, puede saltarse este paso y borrar la célula.

In [ ]:
from sklearn.preprocessing import LabelEncoder

objective_variable = "risk"

le = LabelEncoder()
df[objective_variable] = le.fit_transform(df[objective_variable])
df[objective_variable].unique()

In [ ]:
drop_first_columns = [col for col in object_columns if len(df[col].unique()) == 2 and col != objective_variable]
not_drop_first = [col for col in object_columns if len(df[col]) != 2 and col != objective_variable]

df = pd.get_dummies(df, columns=not_drop_first, drop_first=False, dtype='int')
df = pd.get_dummies(df, columns=drop_first_columns, drop_first=True, dtype='int')

df.head()

In [ ]:
df.info()